In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score
from unidecode import unidecode
import re
import csv 
from nltk.corpus import stopwords
import torch
from gensim.models import Word2Vec
from tqdm import tqdm 
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
torch.set_flush_denormal(True)
import random_seed_setter

random_seed_setter.set_random_seeds()

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
2024-03-31 09:31:08.901711: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


NumPy random seed set with value: 42


2024-03-31 09:31:08.953011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 09:31:09.929635: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow random seed set with value: 42
PyTorch random seed set with value: 42


In [2]:
word2vec = Word2Vec.load("SG_300_25_20/SG_300_25_20.model").wv

In [3]:
print(word2vec.vector_size)

300


In [4]:
html_tag = r"<\S*>"
invalid_characters = r"[^a-zA-Z0-9 ]"
percentage = r"[0-9]+%"
hashtag = r"#\S*"
numeric = r"[0-9]+"
at = r"@\S*"
address = r"\S+@\S+\.\S+"
link = r"(https?:\/\/|www.)\S+"

def preprocess(s):
    # Remove html tags.
    s = re.sub(html_tag, " ", s)

    # Substitute percents.
    s = re.sub(percentage, " procent ", s)

    # Substitute hashtags.
    s = re.sub(hashtag, " hashtag ", s)

    # Substitute at.
    s = re.sub(at, " entitate ", s)

    # Substitute numbers. 
    s = re.sub(numeric, " numar ", s)

    # Substitute addresses.
    s = re.sub(address, " adresa ", s)

    # Substitute links.
    s = re.sub(link, " link ", s)

    # Remove accents, diacritics.
    s = unidecode(s)

    # Keep only these characters
    s = re.sub(invalid_characters, " ", s)
    return s

In [5]:
romainian_stopwords = set(stopwords.words("romanian"))

def tokenize(s):
    tokens = s.split()
    tokens = list(filter(lambda x: x not in romainian_stopwords, tokens))
    tokens = [s.lower() for s in tokens]
    return tokens

In [6]:
# Converts a list of tokens to a vector.
# Only look at the first num_tokens tokens.
def vectorize(tokens, num_tokens=2):
    vectors = []
    for i in range(num_tokens):
        if i >= len(tokens):
            vectors.append(torch.zeros(word2vec.vector_size))
        else:
            if tokens[i] in word2vec:
                vectors.append(torch.tensor(word2vec[tokens[i]])) 
            else:
                vectors.append(torch.zeros(word2vec.vector_size))
    return torch.cat(vectors, dim=0)

In [7]:
def load_train():
    df = pd.read_csv('train.csv')
    df = df.fillna("")

    positive = sum(df["class"] == 1)
    negative = sum(df["class"] == 0)
    print("Positive to negative ratio: ", positive/negative)

    contents =  [vectorize(tokenize(preprocess(x)), num_tokens=15) for x in tqdm(df["content"])]
    titles = [vectorize(tokenize(preprocess(x)), num_tokens=35) for x in tqdm(df["title"])]

    labels = list(df["class"])

    x = []
    for i in range(len(contents)):
        vec = torch.concat([contents[i], titles[i]], dim=0)
        x.append(vec)

    return x , labels 

In [8]:
x, labels = load_train() 
train_x, test_x, train_labels, test_labels = train_test_split(x, labels, test_size=0.2, shuffle=True)

Positive to negative ratio:  1.7887540996562217


100%|██████████| 70575/70575 [00:17<00:00, 4146.99it/s]


In [9]:
class MyDataset(Dataset):
    def __init__(self, x, labels):
        self.x = x
        self.labels = labels
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index], 1 if self.labels[index] == True else 0

NUM_FEATURES = train_x[0].shape[0]
train_dataset = MyDataset(train_x, train_labels)
test_dataset = MyDataset(test_x, test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True)

In [10]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=20,kernel_size=1500, stride=300)
        self.conv2 = nn.Conv1d(in_channels=20, out_channels=40,kernel_size=3)
        self.layer1 = nn.Linear(1760, 400)
        self.layer2 = nn.Linear(400, 2)
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(0.75)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=1)

    def forward(self,x):
        x = torch.unsqueeze(x, dim=1)

        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout(x)


        x = self.conv2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = torch.flatten(x, start_dim=1)

        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer2(x)
        x = self.softmax(x)

        return x

    def predict(self, x):
        x = self.forward(x)
        return torch.argmax(x, dim=1)

In [15]:
loss_fn = torch.nn.CrossEntropyLoss(weight=torch.tensor([1.7, 1]))
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [16]:
def train_one_epoch():
    model.train(True)
    running_loss = 0

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in tqdm(enumerate(train_dataloader)):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()

        if i % 40 == 39:
            print(f"Running loss: {running_loss/(i+1)}") 

    return running_loss/(i+1)

In [17]:
def test_model():
    model.eval()

    predictions = []
    targets = []

    with torch.no_grad():
        running_vloss = 0.0

        for i, vdata in enumerate(test_dataloader):
            x , labels = vdata
            outputs = model(x)

            vloss = loss_fn(outputs, labels)
            running_vloss += vloss.item()

            current_predictions = model.predict(x) 
            predictions+=current_predictions
            targets += labels

        avg_vloss = running_vloss / (i + 1)

        precision = precision_score(targets, predictions)
        recall = recall_score(targets, predictions)

        return avg_vloss, balanced_accuracy_score(targets, predictions), (precision, recall) 

In [18]:
EPOCHS = 80 

best_accuracy = 0

for epoch in range(EPOCHS):
    train_loss = train_one_epoch()
    test_loss, test_accuracy, test_pr = test_model()

    print(f"{epoch}: Train loss: {train_loss}, Test loss: {test_loss}, Test accuracy: {test_accuracy}, Test pr: {test_pr}")

    if test_accuracy > best_accuracy:
        print("Current best!")
        torch.save(model, "best_model.pt")
        best_accuracy = test_accuracy



202it [00:14, 13.10it/s]

Running loss: 0.3671245801448822


221it [00:15, 13.88it/s]


38: Train loss: 0.3668378765496733, Test loss: 0.3658761850425175, Test accuracy: 0.944423692833017, Test pr: (0.9639388188009378, 0.9527698079894064)


42it [00:02, 14.09it/s]

...

54: Train loss: 0.363126572170948, Test loss: 0.3630347517984254, Test accuracy: 0.9475080478859643, Test pr: (0.9656671497046038, 0.9559699845508718)


42it [00:02, 14.70it/s]

Running loss: 0.36570002138614655


57it [00:03, 14.65it/s]


KeyboardInterrupt: 

In [19]:
model = Model()
model = torch.load("best_model.pt")
print(test_model())

(0.3633933344057628, 0.9478906290299214, (0.9667039106145251, 0.9547561244758331))


In [21]:
def load_test():
    df = pd.read_csv('test.csv')
    df = df.fillna("")

    contents =  [vectorize(tokenize(preprocess(x)), num_tokens=15) for x in tqdm(df["content"])]
    titles = [vectorize(tokenize(preprocess(x)), num_tokens=35) for x in tqdm(df["title"])]

    x = []
    for i in range(len(contents)):
        vec = torch.concat([contents[i], titles[i]], dim=0)
        x.append(vec)

    return x 

In [22]:
x_test = load_test() 

100%|██████████| 36669/36669 [00:16<00:00, 2202.34it/s]


In [23]:

ids = []
predictions = []
model.eval()
with torch.no_grad():
    for i in tqdm(range(len(x_test))):
        id = i
        prediction = model.predict(torch.unsqueeze(x_test[i], 0)).item()

        ids.append(id)
        predictions.append(prediction)


df = pd.DataFrame(data={"id":ids, "class": predictions})
print(df)

100%|██████████| 36669/36669 [00:23<00:00, 1536.57it/s]

          id  class
0          0      0
1          1      0
2          2      0
3          3      0
4          4      0
...      ...    ...
36664  36664      0
36665  36665      0
36666  36666      0
36667  36667      1
36668  36668      0

[36669 rows x 2 columns]


In [99]:
df.to_csv("29.csv", index=False)